# Predicting valve position on the basis of liquid level
In this notebook, we will be trying to predict the position, the regulatory valve should assume on the basis of liquid level
## 1. Problem definition
> How well can the valve position be predicted
## 2. Data
> We have two sets of data, the dataset for valve position as well as the liquid level
* The valve position dataset provides the valve position in terms of percentage (0-100)
* The liquid level dataset provides liquid level in the tank in mililitres (ml)
* We are to combine these two separate datasets in order to form our training, validation and test datasets
### 2.1. How to get the train, validation and test data sets
* Firstly we will be combining the valve position and liquid level data from the `Процесс 2_Уровень_008.KIP1.L_S1_2_month.csv` and `Процесс_2_Положение_клпапна_008_KIP1_Pos_KlR7_2_1month.csv` data sets
* Then we will be using the train_test_split to split the combined data in the ratio of 75:25 to get our training and validation datasets
* As for the test data, we will use `Процесс_3_Уровень` data set as the test data set
## 3. Evaluation and Improvisation
We will be using RMSLE (Root Mean Squared Log Error) as well as other evaluation methods. 
The goal of the regression metrics will be to minimize the error.

In [2]:
# Import the required libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
%matplotlib inline

In [3]:
# Read the Process_2_liquid_level 
liquid_level_data = pd.read_csv("Процесс 2_Уровень_008.KIP1.L_S1_2_month.csv", low_memory=False)
# Read the Process_2_Valve_Pos dataset
valve_pos_data = pd.read_csv("Процесс_2_Положение_клпапна_008_KIP1_Pos_KlR7_2_1month.csv", low_memory=False)


In [4]:
liquid_level_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   DateTime  518460 non-null  object 
 1   Vals      518460 non-null  object 
 2   Fracs     514801 non-null  float64
dtypes: float64(1), object(2)
memory usage: 11.9+ MB


In [5]:
len(liquid_level_data.DateTime)

518460

In [6]:
valve_pos_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype  
---  ------    --------------   -----  
 0   DateTime  518460 non-null  object 
 1   Vals      518460 non-null  object 
 2   Fracs     517144 non-null  float64
dtypes: float64(1), object(2)
memory usage: 11.9+ MB


In [7]:
len(valve_pos_data.DateTime)

518460

In [8]:
df1 = liquid_level_data.copy()

In [9]:
df1["DateTime"] = pd.to_datetime(df1["DateTime"], format="%d.%m.%Y %H:%M")

In [10]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   DateTime  518460 non-null  datetime64[ns]
 1   Vals      518460 non-null  object        
 2   Fracs     514801 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 11.9+ MB


In [11]:
df1.DateTime[3]

Timestamp('2023-10-01 00:16:00')

In [12]:
df2 = valve_pos_data.copy()
df2["DateTime"] = pd.to_datetime(df2["DateTime"], format = "%d.%m.%Y %H:%M")

In [13]:
df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   DateTime  518460 non-null  datetime64[ns]
 1   Vals      518460 non-null  object        
 2   Fracs     517144 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 11.9+ MB


In [14]:
df1.isna().sum()

DateTime       0
Vals           0
Fracs       3659
dtype: int64

In [15]:
df2.isna().sum()

DateTime       0
Vals           0
Fracs       1316
dtype: int64

In [16]:
# Check for string dtype
for label, content in df2.items():
    if pd.api.types.is_string_dtype(content):
        print("There are string dtypes in:", label)
    if pd.api.types.is_float_dtype(content):
        print("There are floats in:", label)
    
        

There are string dtypes in: Vals
There are floats in: Fracs


In [17]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 3 columns):
 #   Column    Non-Null Count   Dtype         
---  ------    --------------   -----         
 0   DateTime  518460 non-null  datetime64[ns]
 1   Vals      518460 non-null  object        
 2   Fracs     514801 non-null  float64       
dtypes: datetime64[ns](1), float64(1), object(1)
memory usage: 11.9+ MB


In [18]:
df1["Vals"] = pd.to_numeric(df1["Vals"],errors="coerce")
df2["Vals"] = pd.to_numeric(df2["Vals"],errors="coerce")

In [19]:
#Fill up null values in the columns
for label, content in df1.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            # make a label for if there was missing content or not
            df1[label+" is missing"] = pd.isnull(content)
            # Fill the missing values with 0
            df1[label] = content.fillna(0)
df1["Fracs"] = df1["Fracs"].astype('int64')
df1["Vals"] = df1["Vals"].astype('int64')            

In [20]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DateTime          518460 non-null  datetime64[ns]
 1   Vals              518460 non-null  int64         
 2   Fracs             518460 non-null  int64         
 3   Vals is missing   518460 non-null  bool          
 4   Fracs is missing  518460 non-null  bool          
dtypes: bool(2), datetime64[ns](1), int64(2)
memory usage: 12.9 MB


In [21]:
df1.isna().sum()

DateTime            0
Vals                0
Fracs               0
Vals is missing     0
Fracs is missing    0
dtype: int64

In [22]:
df1.head()

,DateTime,Vals,Fracs,Vals is missing,Fracs is missing
0,2023-10-01 00:16:00,805,4680176,False,False
1,2023-10-01 00:16:00,807,92041,False,False
2,2023-10-01 00:16:00,807,2039795,False,False
3,2023-10-01 00:16:00,807,7359619,False,False
4,2023-10-01 00:16:00,808,9680176,False,False


In [23]:

for label, content in df2.items():
    if pd.api.types.is_numeric_dtype(content):
        if pd.isnull(content).sum():
            # make a label for if there was missing content or not
            df2[label+" is missing"] = pd.isnull(content)
            # Fill the missing values with 0
            df2[label] = content.fillna(0)
df2["Fracs"] = df2["Fracs"].astype('int64')
df2["Vals"] = df2["Vals"].astype('int64')

In [24]:
df2.info()

df2.isna().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DateTime          518460 non-null  datetime64[ns]
 1   Vals              518460 non-null  int64         
 2   Fracs             518460 non-null  int64         
 3   Vals is missing   518460 non-null  bool          
 4   Fracs is missing  518460 non-null  bool          
dtypes: bool(2), datetime64[ns](1), int64(2)
memory usage: 12.9 MB


DateTime            0
Vals                0
Fracs               0
Vals is missing     0
Fracs is missing    0
dtype: int64

In [25]:
df2.head()

,DateTime,Vals,Fracs,Vals is missing,Fracs is missing
0,2023-10-01 00:16:00,1,24000001,False,False
1,2023-10-01 00:16:00,1,24000001,False,False
2,2023-10-01 00:16:00,1,24000001,False,False
3,2023-10-01 00:16:00,1,228000045,False,False
4,2023-10-01 00:16:00,1,24000001,False,False


In [26]:
#df1["Values"] = df1["Vals"]+"."+df1["Fracs"].astype(str)
#df2["Values"] = df2["Vals"]+"."+df2["Fracs"].astype(str)

In [27]:
df1.head(), df2.head()

(             DateTime  Vals    Fracs  Vals is missing  Fracs is missing
 0 2023-10-01 00:16:00   805  4680176            False             False
 1 2023-10-01 00:16:00   807    92041            False             False
 2 2023-10-01 00:16:00   807  2039795            False             False
 3 2023-10-01 00:16:00   807  7359619            False             False
 4 2023-10-01 00:16:00   808  9680176            False             False,
              DateTime  Vals      Fracs  Vals is missing  Fracs is missing
 0 2023-10-01 00:16:00     1   24000001            False             False
 1 2023-10-01 00:16:00     1   24000001            False             False
 2 2023-10-01 00:16:00     1   24000001            False             False
 3 2023-10-01 00:16:00     1  228000045            False             False
 4 2023-10-01 00:16:00     1   24000001            False             False)

In [28]:
df1.info(), df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DateTime          518460 non-null  datetime64[ns]
 1   Vals              518460 non-null  int64         
 2   Fracs             518460 non-null  int64         
 3   Vals is missing   518460 non-null  bool          
 4   Fracs is missing  518460 non-null  bool          
dtypes: bool(2), datetime64[ns](1), int64(2)
memory usage: 12.9 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DateTime          518460 non-null  datetime64[ns]
 1   Vals              518460 non-null  int64         
 2   Fracs             518460 non-null  int64         
 3   Vals is missing   518460 non-null  bool         

(None, None)

In [29]:
time_stamp_match = list(df1["DateTime"] == df2["DateTime"])

In [30]:
time_stamp_match.count(True)

518460

In [31]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 5 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DateTime          518460 non-null  datetime64[ns]
 1   Vals              518460 non-null  int64         
 2   Fracs             518460 non-null  int64         
 3   Vals is missing   518460 non-null  bool          
 4   Fracs is missing  518460 non-null  bool          
dtypes: bool(2), datetime64[ns](1), int64(2)
memory usage: 12.9 MB


In [32]:
df1["Vals is missing"].value_counts()

False    518291
True        169
Name: Vals is missing, dtype: int64

In [33]:
df2["Vals is missing"].value_counts()

False    518291
True        169
Name: Vals is missing, dtype: int64

In [34]:
df1["Fracs is missing"].value_counts(), df2["Fracs is missing"].value_counts()

(False    514801
 True       3659
 Name: Fracs is missing, dtype: int64,
 False    517144
 True       1316
 Name: Fracs is missing, dtype: int64)

In [35]:
df1.head()

,DateTime,Vals,Fracs,Vals is missing,Fracs is missing
0,2023-10-01 00:16:00,805,4680176,False,False
1,2023-10-01 00:16:00,807,92041,False,False
2,2023-10-01 00:16:00,807,2039795,False,False
3,2023-10-01 00:16:00,807,7359619,False,False
4,2023-10-01 00:16:00,808,9680176,False,False


In [36]:
df2.head()

,DateTime,Vals,Fracs,Vals is missing,Fracs is missing
0,2023-10-01 00:16:00,1,24000001,False,False
1,2023-10-01 00:16:00,1,24000001,False,False
2,2023-10-01 00:16:00,1,24000001,False,False
3,2023-10-01 00:16:00,1,228000045,False,False
4,2023-10-01 00:16:00,1,24000001,False,False


In [37]:
df1["Liquid_levels"] = df1["Vals"].astype(str)+"."+df1["Fracs"].astype(str)

In [38]:
df2["Valve_positions"] = df2["Vals"].astype(str)+"."+df2["Fracs"].astype(str)

In [39]:
df1.head()

,DateTime,Vals,Fracs,Vals is missing,Fracs is missing,Liquid_levels
0,2023-10-01 00:16:00,805,4680176,False,False,805.4680176
1,2023-10-01 00:16:00,807,92041,False,False,807.92041
2,2023-10-01 00:16:00,807,2039795,False,False,807.2039795
3,2023-10-01 00:16:00,807,7359619,False,False,807.7359619
4,2023-10-01 00:16:00,808,9680176,False,False,808.9680176


In [40]:
df2.head()

,DateTime,Vals,Fracs,Vals is missing,Fracs is missing,Valve_positions
0,2023-10-01 00:16:00,1,24000001,False,False,1.24000001
1,2023-10-01 00:16:00,1,24000001,False,False,1.24000001
2,2023-10-01 00:16:00,1,24000001,False,False,1.24000001
3,2023-10-01 00:16:00,1,228000045,False,False,1.228000045
4,2023-10-01 00:16:00,1,24000001,False,False,1.24000001


In [41]:
df1["Liquid_levels"] = pd.to_numeric(df1["Liquid_levels"], downcast="float")

In [42]:
df2["Valve_positions"] = pd.to_numeric(df2["Valve_positions"], downcast="float")

In [43]:
df1.info(), df2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DateTime          518460 non-null  datetime64[ns]
 1   Vals              518460 non-null  int64         
 2   Fracs             518460 non-null  int64         
 3   Vals is missing   518460 non-null  bool          
 4   Fracs is missing  518460 non-null  bool          
 5   Liquid_levels     518460 non-null  float32       
dtypes: bool(2), datetime64[ns](1), float32(1), int64(2)
memory usage: 14.8 MB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 6 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DateTime          518460 non-null  datetime64[ns]
 1   Vals              518460 non-null  int64         
 2   Fracs             518460 non-null  i

(None, None)

In [44]:
df1["Liquid_levels"].isnull().sum()

0

In [45]:
df2["Valve_positions"].isnull().sum()

0

In [46]:
df1["Liquid_levels"][df1["Liquid_levels"]>1000] = df1["Liquid_levels"].median()

C:\Users\user\AppData\Local\Temp\ipykernel_12764\2447413430.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["Liquid_levels"][df1["Liquid_levels"]>1000] = df1["Liquid_levels"].median()


In [47]:
df2["Valve_positions"][df2["Valve_positions"]>100]

191768    250.139999
414317    250.139999
Name: Valve_positions, dtype: float32

In [48]:
df2["Valve_positions"][df2["Valve_positions"]>100] = df2["Valve_positions"].median()

C:\Users\user\AppData\Local\Temp\ipykernel_12764\634717652.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df2["Valve_positions"][df2["Valve_positions"]>100] = df2["Valve_positions"].median()


In [49]:
df2["Valve_positions"][df2["Valve_positions"]>100]

Series([], Name: Valve_positions, dtype: float32)

In [50]:
df1["Liquid_levels"].max(), df2["Valve_positions"].max()

(891.360107421875, 59.847999572753906)

In [51]:
np.argmax(df1["Liquid_levels"]), np.argmax(df2.Valve_positions)

(179568, 190374)

In [52]:
df1["Valve_positions"] = df2["Valve_positions"]

In [53]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 7 columns):
 #   Column            Non-Null Count   Dtype         
---  ------            --------------   -----         
 0   DateTime          518460 non-null  datetime64[ns]
 1   Vals              518460 non-null  int64         
 2   Fracs             518460 non-null  int64         
 3   Vals is missing   518460 non-null  bool          
 4   Fracs is missing  518460 non-null  bool          
 5   Liquid_levels     518460 non-null  float32       
 6   Valve_positions   518460 non-null  float32       
dtypes: bool(2), datetime64[ns](1), float32(2), int64(2)
memory usage: 16.8 MB


In [54]:
df1["Year"] = df1.DateTime.dt.year
df1["Month"] = df1.DateTime.dt.month
df1["Day"] = df1.DateTime.dt.day
df1["Hour"] = df1.DateTime.dt.hour
df1["Minute"] = df1.DateTime.dt.minute
df1["DayOfWeek"] = df1.DateTime.dt.dayofweek
df1["DayOfYear"] = df1.DateTime.dt.dayofyear

In [55]:
df1.drop(columns = ["DateTime", "Vals", "Fracs"], inplace = True)

In [56]:
df1.drop(columns = ["Vals is missing", "Fracs is missing"], inplace = True)

In [57]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 518460 entries, 0 to 518459
Data columns (total 9 columns):
 #   Column           Non-Null Count   Dtype  
---  ------           --------------   -----  
 0   Liquid_levels    518460 non-null  float32
 1   Valve_positions  518460 non-null  float32
 2   Year             518460 non-null  int64  
 3   Month            518460 non-null  int64  
 4   Day              518460 non-null  int64  
 5   Hour             518460 non-null  int64  
 6   Minute           518460 non-null  int64  
 7   DayOfWeek        518460 non-null  int64  
 8   DayOfYear        518460 non-null  int64  
dtypes: float32(2), int64(7)
memory usage: 31.6 MB


## Modelling

Preprocessing the data has been completed. Now its time to start the modelling process. 
In order to model the data, we will use sklearn's `train_test_split()` method to split the data between training and validation sets.

In [59]:
from sklearn.model_selection import train_test_split

Now split the dataset between training and validation sets. In this step we will use the liquid level data to predict valve positions
The splitting will be done in a 75:25 ratio. I.e. 75% of the data will be used to train the model and the rest will be used to validate

In [61]:
# First split the dataframe between X and y
X = df1.drop("Valve_positions", axis = 1) 
y = df1["Valve_positions"]

In [62]:
# After creating X and y, its time to split them into training and validation datasets
X_train, X_val, y_train, y_val = train_test_split(X,y,
                                                  test_size=0.25)

We will use the `RandomForestRegressor` to create our model with `n_jobs` = -1 and `random_state` = 54

In [64]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_jobs = -1,
                             random_state=54)

In [65]:
%%time
# Try to fit the model
model.fit(X_train, y_train)

Wall time: 41.2 s


RandomForestRegressor(n_jobs=-1, random_state=54)

In [66]:
model.score(X_train, y_train)

0.8742490893443037

In [67]:
model.score(X_val, y_val)

0.10376724560720163

### Improvisation
Now that we have gotten a score of about **10%** on the validation dataset, it is obvious that we need to improve the model, in order to do that, lets optimize its parameters

Improvisation by optimizing the hyperparameters using `RandomizedSearchCV`

In [70]:
%%time
from sklearn.model_selection import RandomizedSearchCV
rf_grid = {"n_estimators":np.arange(10, 1000, 10),
          "max_depth":[None, 3, 5, 10],
          "min_samples_split": np.arange(2, 40, 2),
          "min_samples_leaf":np.arange(1, 20, 2),
          "max_features":[0.5, 1, "sqrt", "auto"],
          "max_samples": [10000]}

# Instantiate model using these hyperparameters
rs_model = RandomizedSearchCV(RandomForestRegressor(n_jobs=-1,
                                                   random_state=60),
                             param_distributions=rf_grid,
                             n_iter=6,
                             cv=5,
                             verbose=True)


# Fit the model
rs_model.fit(X_train, y_train)

Fitting 5 folds for each of 6 candidates, totalling 30 fits
Wall time: 2min 13s


RandomizedSearchCV(cv=5,
                   estimator=RandomForestRegressor(n_jobs=-1, random_state=60),
                   n_iter=6,
                   param_distributions={'max_depth': [None, 3, 5, 10],
                                        'max_features': [0.5, 1, 'sqrt',
                                                         'auto'],
                                        'max_samples': [10000],
                                        'min_samples_leaf': array([ 1,  3,  5,  7,  9, 11, 13, 15, 17, 19]),
                                        'min_samples_split': array([ 2,  4,  6,  8, 10, 12, 14, 16, 18, 20, 22, 24, 26, 28, 30, 32, 34,
       36, 38]),
                                        'n_estimators': ar...
       140, 150, 160, 170, 180, 190, 200, 210, 220, 230, 240, 250, 260,
       270, 280, 290, 300, 310, 320, 330, 340, 350, 360, 370, 380, 390,
       400, 410, 420, 430, 440, 450, 460, 470, 480, 490, 500, 510, 520,
       530, 540, 550, 560, 570, 580, 590, 600, 610, 

In [71]:
rs_model.best_params_

{'n_estimators': 600,
 'min_samples_split': 20,
 'min_samples_leaf': 13,
 'max_samples': 10000,
 'max_features': 0.5,
 'max_depth': 10}

In [72]:
rs_model.score(X_val, y_val)

0.21854660866234987

In [73]:
# Let's do a function for scoring 
from sklearn.metrics import mean_squared_log_error, mean_absolute_error, r2_score
# Create a function to calculate root mean squared log error 
def rmsle(y_true, y_preds):
    """
    This function takes true labels (y_true) and prediction labels (y_preds) as input and computes the root mean squared log error
    """
    return np.sqrt(mean_squared_log_error(y_true, y_preds))
# Another function to show the various scores using the true labels and prediction labels
def show_scores(model):
    train_preds = model.predict(X_train)
    val_preds = model.predict(X_val)
    scores = {"Training R^2 Score":r2_score(y_train, train_preds),
             "Validation R^2 Score": r2_score(y_val, val_preds),
              "Training Mean Absolute Error": mean_absolute_error(y_train, train_preds),
              "Validation Mean Absolute Error": mean_absolute_error(y_val, val_preds), 
              "Traininig RMSLE": rmsle(y_train, train_preds),
              "Validation RMSLE": rmsle(y_val, val_preds)
             }
    return scores

In [74]:
show_scores(model)

{'Training R^2 Score': 0.8742490893443037,
 'Validation R^2 Score': 0.10376724560720163,
 'Training Mean Absolute Error': 1.8998354126640882,
 'Validation Mean Absolute Error': 5.122229592788022,
 'Traininig RMSLE': 0.6107763327756363,
 'Validation RMSLE': 1.1717012506561009}

In [75]:
show_scores(rs_model)

{'Training R^2 Score': 0.2218583036225409,
 'Validation R^2 Score': 0.21854660866234987,
 'Training Mean Absolute Error': 4.820302965864341,
 'Validation Mean Absolute Error': 4.802343032821837,
 'Traininig RMSLE': 1.0956561576658899,
 'Validation RMSLE': 1.0971213930199375}